In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [111]:
toronto_df = pd.read_csv('canada_postals_with_geo.csv')
toronto_df.rename(columns = {'Neighbourhood': 'Neighborhood'}, inplace = True)
toronto_df.drop(toronto_df.columns[0], axis = 1, inplace = True)
toronto_df.head()


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [112]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto City are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto City are 43.653963, -79.387207.


In [113]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        CLIENT_ID = 'THERX5ZXEOIIN4ATFWWHEL32SEBDAXER100IG2XRYYNM0HFT' # your Foursquare ID
        CLIENT_SECRET = '3MXGRVNPBFJGCVLWRWYYRNCXN4F22Q01FICKDWOHGWJC0GVY' # your Foursquare Secret
        VERSION = '20180605' # Foursquare API version
        LIMIT = 100
        query = 'University'
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            query,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [114]:
toronto_venues = getNearbyVenues(names = toronto_df['Neighborhood'],
                                   latitudes = toronto_df['Latitude'],
                                   longitudes = toronto_df['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [115]:
print(toronto_venues.shape)
toronto_venues.head()

(79, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,York Mills West,43.752758,-79.400049,St. Andrew's Junior High School,43.753880,-79.404208,University
1,Lawrence Park,43.728020,-79.388790,"Glendon Campus, York University",43.726981,-79.394296,University
2,Rosedale,43.679563,-79.377529,U of T President,43.681164,-79.374968,University
3,Church and Wellesley,43.665860,-79.383160,CES - Capital English Solutions,43.664464,-79.384109,University
4,Church and Wellesley,43.665860,-79.383160,205 Richmond (OCADU),43.663597,-79.381882,University


In [116]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",5,5,5,5,5,5
Central Bay Street,9,9,9,9,9,9
Church and Wellesley,3,3,3,3,3,3
"Commerce Court, Victoria Hotel",4,4,4,4,4,4
"Design Exchange, Toronto Dominion Centre",4,4,4,4,4,4
"Dovercourt Village, Dufferin",1,1,1,1,1,1
"First Canadian Place, Underground city",4,4,4,4,4,4
"Forest Hill North, Forest Hill West",1,1,1,1,1,1
"Harbord, University of Toronto",17,17,17,17,17,17


In [117]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 9 uniques categories.


In [118]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,College Academic Building,College Administrative Building,College Arts Building,College Library,College Residence Hall,College Science Building,Community College,Office,University
0,York Mills West,0,0,0,0,0,0,0,0,1
1,Lawrence Park,0,0,0,0,0,0,0,0,1
2,Rosedale,0,0,0,0,0,0,0,0,1
3,Church and Wellesley,0,0,0,0,0,0,0,0,1
4,Church and Wellesley,0,0,0,0,0,0,0,0,1


In [119]:
toronto_onehot.shape

(79, 10)

In [120]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,College Academic Building,College Administrative Building,College Arts Building,College Library,College Residence Hall,College Science Building,Community College,Office,University
0,"Adelaide, King, Richmond",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
1,Central Bay Street,0.222222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.777778
2,Church and Wellesley,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
3,"Commerce Court, Victoria Hotel",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
4,"Design Exchange, Toronto Dominion Centre",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000


In [121]:
toronto_grouped.shape

(16, 10)

In [122]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                             venue  freq
0                       University   1.0
1        College Academic Building   0.0
2  College Administrative Building   0.0
3            College Arts Building   0.0
4                  College Library   0.0


----Central Bay Street----
                             venue  freq
0                       University  0.78
1        College Academic Building  0.22
2  College Administrative Building  0.00
3            College Arts Building  0.00
4                  College Library  0.00


----Church and Wellesley----
                             venue  freq
0                       University   1.0
1        College Academic Building   0.0
2  College Administrative Building   0.0
3            College Arts Building   0.0
4                  College Library   0.0


----Commerce Court, Victoria Hotel----
                             venue  freq
0                       University   1.0
1        College Academic Building   0.0
2  Co

In [123]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [124]:
num_top_venues = 9

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
0,"Adelaide, King, Richmond",University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
1,Central Bay Street,University,College Academic Building,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building
2,Church and Wellesley,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
3,"Commerce Court, Victoria Hotel",University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
4,"Design Exchange, Toronto Dominion Centre",University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building


In [136]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:9] 

array([1, 0, 1, 1, 1, 1, 1, 3, 4], dtype=int32)

In [149]:
# add clustering labels
if 'Cluster Labels' in neighborhoods_venues_sorted.columns:
    neighborhoods_venues_sorted.drop(['Cluster Labels'], axis = 1, inplace = True)
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
else:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


toronto_merged = toronto_df

# merge toronto_grouped with toronto_df to add latitude/longitude for each neighborhood
toronto_merged = pd.merge(toronto_merged, neighborhoods_venues_sorted, on='Neighborhood')

toronto_merged

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
0,M2P,North York,York Mills West,43.752758,-79.400049,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
1,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
2,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
3,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,University,College Academic Building,College Residence Hall,College Arts Building,Office,Community College,College Science Building,College Library,College Administrative Building
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Community College,University,Office,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,University,College Academic Building,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
8,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building


In [141]:
neighborhoods_venues_sorted['Cluster Labels'].unique()

array([1, 0, 3, 4, 2])

In [142]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = '#ffb360',
        fill = True,
        fill_color = '#ff0000',
#         fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


In [143]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[6] + list(range(5, toronto_merged.shape[1]))]]

,1st Most Common Venue,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
4,University,0,University,College Academic Building,College Residence Hall,College Arts Building,Office,Community College,College Science Building,College Library,College Administrative Building
6,University,0,University,College Academic Building,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building


In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
0,North York,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
1,Central Toronto,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
2,Downtown Toronto,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
3,Downtown Toronto,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
7,Downtown Toronto,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
8,Downtown Toronto,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
9,Downtown Toronto,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
11,Central Toronto,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
13,Downtown Toronto,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building
14,West Toronto,1,University,Office,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building


In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
5,Downtown Toronto,2,Community College,University,Office,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building


In [146]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
10,Central Toronto,3,Office,University,Community College,College Science Building,College Residence Hall,College Library,College Arts Building,College Administrative Building,College Academic Building


In [147]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
12,Downtown Toronto,4,University,College Science Building,College Library,College Administrative Building,Office,Community College,College Residence Hall,College Arts Building,College Academic Building
